In [1]:
# CPU版FAISSをインストール
!pip install faiss-cpu

!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install pypdf

!pip install unstructured
!pip install unstructured[local-inference]



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
zsh:1: no matches found: unstructured[local-inference]


In [5]:
# ▼ 2. パスの設定
import os

BASE_DIR = os.path.abspath("")  # 現在のノートブックのディレクトリ
DATA_DIR = os.path.join(BASE_DIR, "data")
STORAGE_DIR = os.path.join(BASE_DIR, "storage")

print("DATA_DIR:", DATA_DIR)
print("STORAGE_DIR:", STORAGE_DIR)

os.makedirs(STORAGE_DIR, exist_ok=True)

# ▼ 3. データ読み込み
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(DATA_DIR)
documents = loader.load()

print(f" {len(documents)} 件のドキュメントを読み込みました")


# ▼ 4. テキスト分割
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
split_texts = splitter.split_documents(documents)

print(f"{len(split_texts)} 個のチャンクに分割されました")


# ▼ 5. 埋め込みモデルを読み込む
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
print("埋め込みモデル読み込み完了")

# ▼ 6. FAISSインデックス作成
from langchain_community.vectorstores.faiss import FAISS

index = FAISS.from_documents(documents=split_texts, embedding=embedding_model)
print("FAISSインデックス作成完了")

# ▼ 8. インデックス保存
index.save_local(folder_path=STORAGE_DIR)
print(f"✅ インデックスを {STORAGE_DIR} に保存しました")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


DATA_DIR: /Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/data
STORAGE_DIR: /Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/storage
 1 件のドキュメントを読み込みました
8 個のチャンクに分割されました
埋め込みモデル読み込み完了
FAISSインデックス作成完了
✅ インデックスを /Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/storage に保存しました


## ▼ 7. 検索テスト

In [4]:
query = "RAGとは何ですか？"
results = index.similarity_search(query, k=2)

for i, doc in enumerate(results, 1):
    print(f"\n--- 類似チャンク {i} ---")
    print(doc.page_content[:200])
    print(f"メタデータ: {doc.metadata}")


--- 類似チャンク 1 ---
## RAG（検索拡張生成）について

huggingfaceなどからllmをダウンロードしてそのままチャットに利用した際、参照する情報はそのllmの学習当時のものとなります。（当たり前ですが）学習していない会社の社内資料や個人用PCのローカルなテキストなどはllmの知識にありません。

このような存在しない情報をllmに与える（参照させる）手法がRAGです。
メタデータ: {'source': '/Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/data/sample.txt'}

--- 類似チャンク 2 ---
イメージ的には以下のような感じです。

・ファインチューニング: 新しい情報を勉強させる。

・RAG: 新しい情報が記載された本を持たせる。

今回は比較的手軽にできるRAGを使用します。

## RAGの手順

RAGの手順としては以下のようになっています。1,2は準備、3～5がチャットを行う際の処理手順です。
メタデータ: {'source': '/Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/data/sample.txt'}


In [6]:
!pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 1. 必要なライブラリのインポート
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

print("ライブラリインポート完了")

# 2. OpenAI APIキーの設定（ローカル環境向けに手動）
import os
from openai import AzureOpenAI
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    endpoint="https://tech0-gpt-australiaeast.openai.azure.com/",
    credential=AzureKeyCredential("OPENAI_API_KEY")
)


# 3. 保存済みFAISSインデックスを読み込み
from langchain_community.vectorstores.faiss import FAISS

embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large"
)

index = FAISS.load_local(
    folder_path=STORAGE_DIR,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
print(f"✅ インデックス読み込み完了（{index.index.ntotal} 個のベクトル）")

# 4. プロンプトテンプレートの作成
question_prompt_template = """以下のコンテキスト情報を参照して、質問に日本語で答えてください。
コンテキストに関連する情報がない場合は、「提供された資料には該当する情報がありません」と回答してください。

コンテキスト:
{context}

質問: {question}

回答: """

QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template,
    input_variables=["context", "question"]
)
print("✅ プロンプトテンプレート作成完了")

# 5. LLM（ChatGPT）設定
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.3,
    max_tokens=500
)
print("✅ LLM設定完了")

# 6. RetrievalQA チェーン作成
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(search_kwargs={'k': 2}),
    chain_type_kwargs={"prompt": QUESTION_PROMPT},
    chain_type="stuff",
    return_source_documents=True
)
print("✅ RetrievalQA チェーン作成完了")

# 7. テスト実行
test_questions = [
    "RAGとは何ですか？",
    "ファインチューニングとRAGの違いは？",
    "RAGの手順を教えてください"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n--- テスト {i} ---")
    print(f"質問: {question}")
    try:
        response = chain.invoke({"query": question})
        print(f"回答: {response['result']}")
        print("参照されたチャンク:")
        for j, source in enumerate(response["source_documents"], 1):
            print(f"   {j}. {source.page_content[:80]}...")
    except Exception as e:
        print(f"❌ エラー: {e}")